<a href="https://colab.research.google.com/github/fjprcosta/ProjetoIndividual/blob/main/Projeto_Individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Conectores e Bibliotecas

In [ ]:
!pip install pyspark

In [ ]:
pip install pymongo[srv]

In [ ]:
pip install gcsfs

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pymongo import MongoClient
import pandas as pd
from google.cloud import storage
import os
import pandas.io.sql as psql
from numpy import NaN
import numpy as np

In [ ]:
#Conectando Mongo
client = MongoClient("mongodb+srv://<username>:<password>@cluster0.mquek.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")

In [ ]:
#autorizando acesso ao bucket gcp através da key
serviceAccount = '/content/drive/MyDrive/Datasets/chave/projeto-soulcode-350816-3ba44a408971.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#lendo dataset direto do bucket
df = pd.read_csv(r'gs://felipesoulcodebc17/originais/marketing_campaign.csv', sep = ";", encoding='UTF-8')

##Transferindo os dados para o Mongo

In [ ]:
#determinando o database
db = client['soulcode']

In [ ]:
#determinando a collection
collection = db['marketing_campaign']

In [ ]:
#transformando em dicionario
df_dicio = df.to_dict("records")

In [ ]:
#inserinido no mongo
collection.insert_many(df_dicio)

##Tratamento com Pandas

In [ ]:
#criando dataframe
dforiginal = pd.json_normalize(collection.find())

In [ ]:
#determinando máximo de colunas a serem exibidas
pd.set_option('display.max_columns',33)

In [ ]:
#exibindo uma primeira linha para análize inicial
dforiginal.head(1)

,_id,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,62913d3ac36c21139f74240e,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1


In [ ]:
#renomenando colunas de inglês para portugues
dforiginal.rename(columns={'Year_Birth': 'Nascimento', 
                           'Education': 'Escolaridade',
                           'Marital_Status':'Estado_Civil',
                           'Income':'Renda',
                           'Kidhome':'Dependentes_crianças',
                           'Teenhome':'Dependentes_adolescentes',
                           'Dt_Customer':'Data_cadastro',
                           'MntWines':'Gasto_em_vinho',
                           'MntFruits':'Gasto_em_frutas',
                           'MntMeatProducts':'Gasto_em_carnes',
                           'MntFishProducts':'Gasto_em_peixe',
                           'MntSweetProducts':'Gasto_em_doces',
                           'MntGoldProds':'Gasto_em_ouro',
                           'NumDealsPurchases':'Compras_negociadas',
                           'NumWebPurchases':'Compras_website',
                           'NumCatalogPurchases':'Compras_por_catálogo',
                           'NumStorePurchases':'Compras_loja',
                           'NumWebVisitsMonth': 'Número_visitas_mês',
                           'Complain':'Reclamação',
                           'Response':'Aceitação'
                           }, inplace=True)

In [ ]:
#traduzindo o estado civil
dforiginal['Estado_Civil'].replace({'Alone':'Soteiro(a)',
                                    'Absurd':'Soteiro(a)',
                                    'YOLO':'Soteiro(a)',
                                    'Single':'Soteiro(a)',
                                    'Together':'União Estável',
                                    'Married':'Casado(a)',
                                    'Divorced':'Divorciado(a)',
                                    'Widow':'Viúvo(a)'
                                    }, inplace=True)

In [ ]:
#traduzindo o nível de escolaridade (usando sintaxe diferente somente para a prática)
dforiginal.replace(to_replace = '2n Cycle', value = 'Licenciatura/Magistrato', inplace=True)
dforiginal.replace(to_replace = 'Basic', value = 'Nível Fundamental', inplace=True)
dforiginal.replace(to_replace = 'Graduation', value = 'Graduação', inplace=True)
dforiginal.replace(to_replace = 'PhD', value = 'Doutorado', inplace=True)
dforiginal.replace(to_replace = 'Master', value = 'Mestrado', inplace=True)

In [ ]:
#trocando os resultados boleanos de números para sim ou não
dforiginal['Aceitação'].replace({0:'Não', 1: 'Sim'}, inplace=True)

In [ ]:
#fazendo nova análise
dforiginal.head(1)

,_id,ID,Nascimento,Escolaridade,Estado_Civil,Renda,Dependentes_crianças,Dependentes_adolescentes,Data_cadastro,Recency,Gasto_em_vinho,Gasto_em_frutas,Gasto_em_carnes,Gasto_em_peixe,Gasto_em_doces,Gasto_em_ouro,Compras_negociadas,Compras_website,Compras_por_catálogo,Compras_loja,Número_visitas_mês,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Reclamação,Z_CostContact,Z_Revenue,Aceitação
0,62913d3ac36c21139f74240e,5524,1957,Graduação,Soteiro(a),58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,Sim


In [ ]:
#dropando colunas que não servirão para posterior insights
dforiginal.drop(['_id','Recency','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2','Z_CostContact','Z_Revenue'],axis=1,inplace=True)

In [ ]:
#transformando coluna data em Date
dforiginal.Data_cadastro = pd.to_datetime(dforiginal.Data_cadastro)

In [ ]:
#conferindo tipos de dados do dataframe
dforiginal.dtypes

ID                                   int64
Nascimento                           int64
Escolaridade                        object
Estado_Civil                        object
Renda                              float64
Dependentes_crianças                 int64
Dependentes_adolescentes             int64
Data_cadastro               datetime64[ns]
Gasto_em_vinho                       int64
Gasto_em_frutas                      int64
Gasto_em_carnes                      int64
Gasto_em_peixe                       int64
Gasto_em_doces                       int64
Gasto_em_ouro                        int64
Compras_negociadas                   int64
Compras_website                      int64
Compras_por_catálogo                 int64
Compras_loja                         int64
Número_visitas_mês                   int64
Reclamação                           int64
Aceitação                           object
dtype: object

In [ ]:
#contando valores nulos
dforiginal.isnull().sum()

ID                           0
Nascimento                   0
Escolaridade                 0
Estado_Civil                 0
Renda                       24
Dependentes_crianças         0
Dependentes_adolescentes     0
Data_cadastro                0
Gasto_em_vinho               0
Gasto_em_frutas              0
Gasto_em_carnes              0
Gasto_em_peixe               0
Gasto_em_doces               0
Gasto_em_ouro                0
Compras_negociadas           0
Compras_website              0
Compras_por_catálogo         0
Compras_loja                 0
Número_visitas_mês           0
Reclamação                   0
Aceitação                    0
dtype: int64

In [ ]:
#preenchendo valores nulos da coluna Renda com a média
dforiginal['Renda'].fillna(float(dforiginal['Renda'].mean()), inplace=True)

In [ ]:
#reduzindo os números depois da vírgula(os números com a média ficavam com 5 digitos após a vírgula)
dforiginal = dforiginal.round({"Renda":2})

In [ ]:
#conferencia final
dforiginal.head(10)

,ID,Nascimento,Escolaridade,Estado_Civil,Renda,Dependentes_crianças,Dependentes_adolescentes,Data_cadastro,Gasto_em_vinho,Gasto_em_frutas,Gasto_em_carnes,Gasto_em_peixe,Gasto_em_doces,Gasto_em_ouro,Compras_negociadas,Compras_website,Compras_por_catálogo,Compras_loja,Número_visitas_mês,Reclamação,Aceitação
0,5524,1957,Graduação,Soteiro(a),58138.0,0,0,2012-09-04,635,88,546,172,88,88,3,8,10,4,7,0,Sim
1,2174,1954,Graduação,Soteiro(a),46344.0,1,1,2014-03-08,11,1,6,2,1,6,2,1,1,2,5,0,Não
2,5324,1981,Doutorado,Casado(a),58293.0,1,0,2014-01-19,173,43,118,46,27,15,5,5,3,6,5,0,Não
3,8180,1952,Mestrado,Divorciado(a),59354.0,1,1,2013-11-15,233,2,53,3,5,14,3,6,1,5,6,0,Não
4,2114,1946,Doutorado,Soteiro(a),82800.0,0,0,2012-11-24,1006,22,115,59,68,45,1,7,6,12,3,0,Sim
5,9736,1980,Graduação,Casado(a),41850.0,1,1,2012-12-24,53,5,19,2,13,4,3,3,0,3,8,0,Não
6,6565,1949,Mestrado,Casado(a),76995.0,0,1,2013-03-28,1012,80,498,0,16,176,2,11,4,9,5,0,Não
7,7892,1969,Graduação,Soteiro(a),18589.0,0,0,2013-01-02,6,4,25,15,12,13,2,2,1,3,7,0,Não
8,10755,1976,Licenciatura/Magistrato,Casado(a),23718.0,1,0,2013-09-02,6,3,14,15,7,36,3,3,1,2,7,0,Não
9,8595,1973,Graduação,Viúvo(a),42429.0,0,1,2014-02-11,55,0,6,2,0,4,2,1,1,3,5,0,Não


In [ ]:
#salvando dataframe pré-tratado em csv no bucket
dforiginal.to_csv('gs://felipesoulcodebc17/originais/marketing_campaign_pre_tratada.csv', index=False)

##Tratamento/Análise com Spark

In [ ]:
#CÓDIGO QUE ACESSA A BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS
client = storage.Client()

#CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('felipesoulcodebc17')
#USAR O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO (JSON, CSV, PARQUET)
bucket.blob('marketing_campaign_pre_tratada.csv')
#CRIA UMA VARIÁVEL PATH PARA COLOCAR O CAMINHO DO CSV
path = 'gs://felipesoulcodebc17/originais/marketing_campaign_pre_tratada.csv'

In [ ]:
spark = (
    SparkSession.builder
                .master ('local')
                .appName('spark-gcs')
                .config('spark.ui.port','4050')
                .config('spark.jars','https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [ ]:
meu_schema = ( StructType ([
                            StructField("ID", IntegerType(), True),
                            StructField("Nascimento", IntegerType(), True),
                            StructField("Escolaridade", StringType(), True),
                            StructField("Estado_Civil", StringType(), True),
                            StructField("Renda", FloatType(), True),
                            StructField("Dependentes_crianças", IntegerType(), True),
                            StructField("Dependentes_adolescentes", IntegerType(), True),
                            StructField("Data_cadastro", StringType(), True),
                            StructField("Gasto_em_vinho", IntegerType(), True),
                            StructField("Gasto_em_frutas", IntegerType(), True),
                            StructField("Gasto_em_carnes", IntegerType(), True),
                            StructField("Gasto_em_peixe", IntegerType(), True),
                            StructField("Gasto_em_doces", IntegerType(), True),
                            StructField("Gasto_em_ouro", IntegerType(), True),
                            StructField("Compras_negociadas",IntegerType(), True),
                            StructField("Compras_website", IntegerType(), True),
                            StructField("Compras_catálogo", IntegerType(), True),
                            StructField("Compras_loja", IntegerType(), True),
                            StructField("Número_visitas_mês", IntegerType(), True),
                            StructField("Reclamação", IntegerType(), True),
                            StructField("Aceitação", StringType(), True),      
                            ])
)

In [ ]:
#criar um dataframe pelo spark
df_spark =   (spark.read.format('csv')
                       .option('delimiter', ',')
                       .option('header', 'true')
                       .option('inferschema', 'true')
                       .load('gs://felipesoulcodebc17/originais/marketing_campaign_pre_tratada.csv', schema=meu_schema)

)

In [ ]:
#lendo dataframe
df_spark.show(1)

+----+----------+------------+------------+-------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+
|  ID|Nascimento|Escolaridade|Estado_Civil|  Renda|Dependentes_crianças|Dependentes_adolescentes|Data_cadastro|Gasto_em_vinho|Gasto_em_frutas|Gasto_em_carnes|Gasto_em_peixe|Gasto_em_doces|Gasto_em_ouro|Compras_negociadas|Compras_website|Compras_catálogo|Compras_loja|Número_visitas_mês|Reclamação|Aceitação|
+----+----------+------------+------------+-------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+
|5524|      1957|   Graduação|  Soteiro(a)|58138.0|                   0|    

In [ ]:
#transformando coluna data em Date
#df_spark1 = df_spark.withColumn("Data_cadastro", df_spark["Data_cadastro"].cast('Date'))

In [ ]:
df_spark.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Nascimento: integer (nullable = true)
 |-- Escolaridade: string (nullable = true)
 |-- Estado_Civil: string (nullable = true)
 |-- Renda: float (nullable = true)
 |-- Dependentes_crianças: integer (nullable = true)
 |-- Dependentes_adolescentes: integer (nullable = true)
 |-- Data_cadastro: string (nullable = true)
 |-- Gasto_em_vinho: integer (nullable = true)
 |-- Gasto_em_frutas: integer (nullable = true)
 |-- Gasto_em_carnes: integer (nullable = true)
 |-- Gasto_em_peixe: integer (nullable = true)
 |-- Gasto_em_doces: integer (nullable = true)
 |-- Gasto_em_ouro: integer (nullable = true)
 |-- Compras_negociadas: integer (nullable = true)
 |-- Compras_website: integer (nullable = true)
 |-- Compras_catálogo: integer (nullable = true)
 |-- Compras_loja: integer (nullable = true)
 |-- Número_visitas_mês: integer (nullable = true)
 |-- Reclamação: integer (nullable = true)
 |-- Aceitação: string (nullable = true)



In [ ]:
df_spark = df_spark.withColumnRenamed("ID","ID_Cliente").withColumnRenamed("Renda","Renda_Familiar_Anual")

In [ ]:
df_spark.groupBy('Escolaridade').count().show()

+--------------------+-----+
|        Escolaridade|count|
+--------------------+-----+
|Licenciatura/Magi...|  203|
|           Doutorado|  486|
|   Nível Fundamental|   54|
|            Mestrado|  370|
|           Graduação| 1127|
+--------------------+-----+



In [ ]:
#relação entre nível de renda e estado civil
df_spark.groupBy('Estado_Civil').agg(F.mean('Renda_Familiar_Anual')).show()

+-------------+-------------------------+
| Estado_Civil|avg(Renda_Familiar_Anual)|
+-------------+-------------------------+
|   Soteiro(a)|        51051.32905544148|
|     Viúvo(a)|        56426.56168831169|
|Divorciado(a)|        52834.22844827586|
|    Casado(a)|         51729.2103587963|
|União Estável|        53233.48577586207|
+-------------+-------------------------+



In [ ]:
#criando coluna com as classificações de renda
df_spark1 = df_spark.withColumn('Nível_de_Renda', F.when((F.col('Renda_Familiar_Anual') > 0) & (F.col('Renda_Familiar_Anual') < 36000), F.lit('Baixa_renda'))
                                                   .when((F.col('Renda_Familiar_Anual') > 36001) & (F.col('Renda_Familiar_Anual') < 60000), F.lit('Classe_média'))
                                                   .otherwise(F.lit('Alta_Renda')))

In [ ]:
#criando coluna com faixa etária (Até 19 jovem/20 a 59 Adulto/60 pra cima Idoso)
df_spark1 = df_spark1.withColumn('Faixa_Etária', F.when((F.col('Nascimento') > 1997) & (F.col('Nascimento') < 2022), F.lit('Jovem'))
                                                  .when((F.col('Nascimento') > 1963) & (F.col('Nascimento') < 2002), F.lit('Adulto'))
                                                  .otherwise(F.lit('Idoso')))

In [ ]:
#criando coluna de total de compras
df_spark1 = df_spark1.withColumn("Total_Compras",(df_spark1["Compras_negociadas"] + 
                                                  df_spark1["Compras_website"] + 
                                                  df_spark1["Compras_catálogo"] + 
                                                  df_spark1["Compras_loja"]))

In [ ]:
df_spark1.show(1)

+----------+----------+------------+------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+--------------+------------+-------------+
|ID_Cliente|Nascimento|Escolaridade|Estado_Civil|Renda_Familiar_Anual|Dependentes_crianças|Dependentes_adolescentes|Data_cadastro|Gasto_em_vinho|Gasto_em_frutas|Gasto_em_carnes|Gasto_em_peixe|Gasto_em_doces|Gasto_em_ouro|Compras_negociadas|Compras_website|Compras_catálogo|Compras_loja|Número_visitas_mês|Reclamação|Aceitação|Nível_de_Renda|Faixa_Etária|Total_Compras|
+----------+----------+------------+------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+------

In [ ]:
#relação entre nível de renda e escolaridade
#Níveis maiores de escolaridade têm maiores salários
df_spark1.groupBy('Escolaridade').agg(F.mean('Renda_Familiar_Anual')).show()

+--------------------+-------------------------+
|        Escolaridade|avg(Renda_Familiar_Anual)|
+--------------------+-------------------------+
|Licenciatura/Magi...|        47701.37807881773|
|           Doutorado|         56105.2103909465|
|   Nível Fundamental|        20306.25925925926|
|            Mestrado|        52908.47635135135|
|           Graduação|         52715.7557675244|
+--------------------+-------------------------+



In [ ]:
#ordenando por Renda
df_spark1.sort(F.col('Renda_Familiar_Anual')).show()

+----------+----------+--------------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+--------------+------------+-------------+
|ID_Cliente|Nascimento|        Escolaridade| Estado_Civil|Renda_Familiar_Anual|Dependentes_crianças|Dependentes_adolescentes|Data_cadastro|Gasto_em_vinho|Gasto_em_frutas|Gasto_em_carnes|Gasto_em_peixe|Gasto_em_doces|Gasto_em_ouro|Compras_negociadas|Compras_website|Compras_catálogo|Compras_loja|Número_visitas_mês|Reclamação|Aceitação|Nível_de_Renda|Faixa_Etária|Total_Compras|
+----------+----------+--------------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+--------------

In [ ]:
#compras na web por estado civil
df_spark1.groupBy('Estado_Civil').agg(F.sum('Compras_website')).show()

+-------------+--------------------+
| Estado_Civil|sum(Compras_website)|
+-------------+--------------------+
|   Soteiro(a)|                1895|
|     Viúvo(a)|                 356|
|Divorciado(a)|                1000|
|    Casado(a)|                3532|
|União Estável|                2367|
+-------------+--------------------+



In [ ]:
#compras por todos os canais por faixa etária
df_spark1.groupBy('Estado_Civil').agg(F.count('Faixa_etária'), F.sum('Compras_website'), F.sum('Compras_catálogo'), F.sum('Compras_loja')).show()

+-------------+-------------------+--------------------+---------------------+-----------------+
| Estado_Civil|count(Faixa_etária)|sum(Compras_website)|sum(Compras_catálogo)|sum(Compras_loja)|
+-------------+-------------------+--------------------+---------------------+-----------------+
|   Soteiro(a)|                487|                1895|                 1267|             2744|
|     Viúvo(a)|                 77|                 356|                  256|              494|
|Divorciado(a)|                232|                1000|                  620|             1350|
|    Casado(a)|                864|                3532|                 2268|             5055|
|União Estável|                580|                2367|                 1552|             3327|
+-------------+-------------------+--------------------+---------------------+-----------------+



In [ ]:
#Criando uma window
w0 = Window.partitionBy(df_spark1["Escolaridade"]).orderBy("Total_Compras")

In [ ]:
#Window Function1 - Ranking
df_spark1.withColumn("Ranking",F.rank().over(w0)).show()

+----------+----------+------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+--------------+------------+-------------+-------+
|ID_Cliente|Nascimento|Escolaridade| Estado_Civil|Renda_Familiar_Anual|Dependentes_crianças|Dependentes_adolescentes|Data_cadastro|Gasto_em_vinho|Gasto_em_frutas|Gasto_em_carnes|Gasto_em_peixe|Gasto_em_doces|Gasto_em_ouro|Compras_negociadas|Compras_website|Compras_catálogo|Compras_loja|Número_visitas_mês|Reclamação|Aceitação|Nível_de_Renda|Faixa_Etária|Total_Compras|Ranking|
+----------+----------+------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---

In [ ]:
#Window Function2 - Dense Rank
df_spark1.withColumn("Dense_Rank",F.dense_rank().over(w0)).show()

+----------+----------+------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+------------------+---------------+----------------+------------+------------------+----------+---------+--------------+------------+-------------+----------+
|ID_Cliente|Nascimento|Escolaridade| Estado_Civil|Renda_Familiar_Anual|Dependentes_crianças|Dependentes_adolescentes|Data_cadastro|Gasto_em_vinho|Gasto_em_frutas|Gasto_em_carnes|Gasto_em_peixe|Gasto_em_doces|Gasto_em_ouro|Compras_negociadas|Compras_website|Compras_catálogo|Compras_loja|Número_visitas_mês|Reclamação|Aceitação|Nível_de_Renda|Faixa_Etária|Total_Compras|Dense_Rank|
+----------+----------+------------+-------------+--------------------+--------------------+------------------------+-------------+--------------+---------------+---------------+--------------+--------------+-------------+----------------

In [ ]:
#Select para exibir o ID_Cliente, Escolaridade, Renda_Familiar_Anual, Total_Compras de clientes ordenado pelo total de compras
df_spark1.select(df_spark1["ID_Cliente"],df_spark1["Escolaridade"],df_spark1["Renda_Familiar_Anual"],df_spark1["Total_Compras"]).orderBy("Total_Compras").show()

+----------+--------------------+--------------------+-------------+
|ID_Cliente|        Escolaridade|Renda_Familiar_Anual|Total_Compras|
+----------+--------------------+--------------------+-------------+
|     11110|           Graduação|              3502.0|            0|
|     11181|           Doutorado|            156924.0|            0|
|      5555|           Graduação|            153924.0|            0|
|      3955|           Graduação|              4861.0|            0|
|      4303|           Doutorado|              6835.0|            1|
|      1503|           Doutorado|            162397.0|            1|
|      9303|           Graduação|              5305.0|            1|
|      8720|Licenciatura/Magi...|            52247.25|            1|
|      5899|           Doutorado|              5648.0|            2|
|      5336|            Mestrado|            157733.0|            2|
|      4246|            Mestrado|              6560.0|            2|
|     10401|Licenciatura/Magi...| 

In [ ]:
df_mongo = df_spark1.toPandas()

In [ ]:
df_mongo.dtypes

ID_Cliente                    int32
Nascimento                    int32
Escolaridade                 object
Estado_Civil                 object
Renda_Familiar_Anual        float32
Dependentes_crianças          int32
Dependentes_adolescentes      int32
Data_cadastro                object
Gasto_em_vinho                int32
Gasto_em_frutas               int32
Gasto_em_carnes               int32
Gasto_em_peixe                int32
Gasto_em_doces                int32
Gasto_em_ouro                 int32
Compras_negociadas            int32
Compras_website               int32
Compras_catálogo              int32
Compras_loja                  int32
Número_visitas_mês            int32
Reclamação                    int32
Aceitação                    object
Nível_de_Renda               object
Faixa_Etária                 object
Total_Compras                 int32
dtype: object

In [ ]:
#inserindo df tratado como .csv no bucket
df_mongo.to_csv('gs://felipesoulcodebc17/tratados/marketing_campaign_tratada1.csv', index=False)

In [ ]:
#determinando a collection
collection1 = db['marketing_campaign_tratada1']

In [ ]:
#transformando em dicionario
df_dicio2 = df_mongo.to_dict("records")

In [ ]:
#inserindo df tratado no mongo
collection1.insert_many(df_dicio2)